<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We use NumPy for math, pandas for working with tables, matplotlib for charting, yfinance for getting financial prices, and SciPy for optimization.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

This code fixes the seed so that we get the same results each time we run the simulation.

In [ ]:
np.random.seed(42)

Here, we define how many stocks we want, how much data to look at, and which companies we’re studying.

In [ ]:
n_assets = 5
n_obs = 750
symbols = ["AAPL", "MSFT", "GOOGL", "META", "NVDA"]

We use yfinance to pull closing stock prices, adjust for any splits/dividends, and calculate daily log returns for our assets over the last three years.

In [ ]:
data = yf.download(
    symbols, period="3y", interval="1d", progress=False, auto_adjust=True
)["Close"].dropna()
returns = np.log(data).diff().dropna().iloc[-n_obs:]
mean_return_annual = returns.mean() * 252
cov_annual = returns.cov() * 252

In this first section, we're building the groundwork for our analysis. We set up our scientific tools, lock in reproducibility, select our stocks, and pull historical price data from Yahoo Finance. Then, we convert those prices to returns, and prepare averages and risk levels we’ll need to assess different portfolios. These calculations ensure all later steps use realistic and consistent data.

## Simulate synthetic return scenarios

Here, we set up numbers to use in case real data is missing, then calculate average returns, risk (volatility), and relationships between each stock. We use these to randomize new returns similar to what we might see in real life. This gives us clean data for building our portfolios.

In [ ]:
drift = 0.10
mu = (
    mean_return_annual.values
    if not np.any(np.isnan(mean_return_annual))
    else np.full(n_assets, drift)
)
sigma = (
    np.sqrt(np.diag(cov_annual))
    if not np.any(np.isnan(cov_annual))
    else np.full(n_assets, 0.20)
)
corr_matrix = (
    cov_annual.corr().values if not np.any(np.isnan(cov_annual)) else np.eye(n_assets)
)
cov = np.outer(sigma, sigma) * corr_matrix
synthetic_returns = np.random.multivariate_normal(mu / 252, cov / 252, size=n_obs)
synthetic_returns = pd.DataFrame(synthetic_returns, columns=symbols)

This block builds a simulated world where every stock's return matches the math from our real-world estimates. If there's missing info, we fall back on rough assumptions so everything stays robust. We end up with a fresh batch of data that mimics real returns, set up in a table just like what you'd get from actual market prices. This synthetic data helps us control for outliers and makes the modeling straightforward and reproducible.

Here, we get rid of any days that look too different from the rest, cleaning the data so a handful of wild numbers don’t throw off our analysis.

In [ ]:
outlier_thresh = 4
mean_return = synthetic_returns.mean()
std_return = synthetic_returns.std()
non_outlier_idx = (
    (synthetic_returns - mean_return).abs() <= outlier_thresh * std_return
).all(axis=1)
filtered_returns = synthetic_returns[non_outlier_idx]

We filter out any unusually large or small simulated returns that fall too far from the average pattern. This helps keep our later results reliable and not influenced by extreme, rare events. After this cleaning, only the most representative return scenarios remain, making our future calculations much more dependable.

From this cleaner data, we recalculate the realistic, annualized average returns and risk for each stock so we can use them in portfolio construction.

In [ ]:
sim_mu = filtered_returns.mean().values * 252
sim_cov = filtered_returns.cov().values * 252

Now, with extreme outliers removed, we calculate new estimates for what the stocks might earn and how their risks interact. We annualize the results so everything is on a common scale for use in later portfolio math. This ensures that our inputs better reflect what an investor might actually experience, without distortion from wild, one-off days.

## Build and evaluate portfolios

We randomly mix the five stocks into 400 different portfolios, then calculate expected return and risk for each one based on our cleaned estimates.

In [ ]:
n_samples = 400
rand_weights = np.random.dirichlet(np.ones(n_assets), size=n_samples)
rand_port_rets = rand_weights @ sim_mu
rand_port_vols = np.sqrt(np.einsum("ij,jk,ik->i", rand_weights, sim_cov, rand_weights))

By creating many random mixes, we can see what’s possible if you just pick different asset weights at random. We calculate how much you could expect to gain and how much volatility you’d take on for each approach. This gives us a full map of possible outcomes across the landscape of diversified portfolios, all based on consistent risk and return numbers.

We prepare the return and risk numbers we just estimated so they can be used for designing portfolios in the next block.

In [ ]:
mat_mu = sim_mu
mat_cov = sim_cov

This step just saves our newly cleaned and annualized averages and risk calculations, which we’ll use as official inputs. This keeps everything tidy and avoids confusion as we try new methods to optimize these portfolios.

Here, we plot out the full range of optimal risk-return combinations (“efficient frontier”) by asking: for each risk level, what’s the best portfolio we can construct?

In [ ]:
risk_aversion_grid = np.linspace(0.0001, 10, 40)
ef_returns = []
ef_vols = []
ef_weights = []
for gamma in risk_aversion_grid:

    def obj(w):
        return -w @ mat_mu + gamma * 0.5 * w @ mat_cov @ w

    cons = [
        {"type": "eq", "fun": lambda w: np.sum(w) - 1},
        {"type": "ineq", "fun": lambda w: w},
    ]
    bounds = [(0, 1) for _ in range(n_assets)]
    res = minimize(
        obj,
        np.ones(n_assets) / n_assets,
        method="SLSQP",
        constraints=cons,
        bounds=bounds,
        options={"disp": False},
    )
    x = res.x
    ef_weights.append(x)
    ef_returns.append(x @ mat_mu)
    ef_vols.append(np.sqrt(x @ mat_cov @ x))

We methodically adjust the balance between wanting more return and wanting less risk. For each point along that trade-off, we use optimization from SciPy to find the best way to split our money across the stocks. Our process always requires we invest every dollar and don’t short sell. This creates a smooth curve of optimal portfolios, showing exactly what’s possible from best to most conservative.

Now we create the “Kelly curve,” a special set of portfolios that aim to maximize how quickly our wealth could grow if returns stay consistent. This method doesn’t limit portfolio weights.

In [ ]:
inv_cov = np.linalg.pinv(mat_cov)
kelly_w = inv_cov @ mat_mu
kelly_scale = np.linspace(0, 2, 80)
kelly_port_returns = [scale * kelly_w @ mat_mu for scale in kelly_scale]
kelly_port_vols = [
    np.sqrt((scale * kelly_w) @ mat_cov @ (scale * kelly_w)) for scale in kelly_scale
]

We use a mathematical shortcut to find one particular way to invest that should maximize growth over the long run. By scaling that approach up and down, we see what happens when you take on more or less risk. Unlike our earlier optimized portfolios, these portfolios ignore any limits on where your money can go, giving us another view on the risk-return space.

## Visualize the portfolio risk and return

We make a chart showing every random portfolio, the optimal line of portfolios, and the Kelly curve. This lets us compare approaches and see how different strategies relate in terms of risk and expected return.

In [ ]:
plt.figure(figsize=(9, 7))
plt.scatter(
    rand_port_vols,
    rand_port_rets,
    c="lightgray",
    label="Random Portfolios",
    s=20,
    alpha=0.7,
)
plt.plot(ef_vols, ef_returns, color="navy", lw=2, label="Efficient Frontier")
plt.plot(kelly_port_vols, kelly_port_returns, color="green", lw=2, label="Kelly Curve")
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Return")
plt.title("Random Portfolios, Efficient Frontier, and Kelly Curve")
plt.legend()
plt.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

Our final step visualizes everything we’ve built. Each gray dot happens when we throw stock picks together at random, while the navy line shows the best you can do for any risk level. The green curve marks the theoretically highest-growth approaches. By comparing the dots, the line, and the curve, we see the range of choices in managing risk versus reward. The plot gives us a clear, direct look at how our portfolio-building strategy shapes real investment possibilities.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.